In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import scipy.signal as ss

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
holidays_events = pd.read_csv("https://www.dropbox.com/s/bxyamlpevkiwwoq/holidays_events.csv?dl=1")
oil = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1",parse_dates=['date'],index_col='date')
oil2 = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1")
sample_submission = pd.read_csv("https://www.dropbox.com/s/68jjl61x6u3klos/sample_submission.csv?dl=1")
stores = pd.read_csv("https://www.dropbox.com/s/lcxn6r9bs2exguq/stores.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/cvdo1gn7r5lu2uz/test.csv?dl=1",index_col='id')
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')
transactions = pd.read_csv("https://www.dropbox.com/s/92fij9bcwt0e0cj/transactions.csv?dl=1")

Wybieramy obserwacje dla family=AUTOMOTIVE

In [44]:
train_automotive = train.loc[(train['family']=='AUTOMOTIVE')]

In [45]:
train_automotive.tail()

,date,store_nbr,family,sales,onpromotion
id,,,,,
3000723,2017-08-15,54,AUTOMOTIVE,8.0,0
3000756,2017-08-15,6,AUTOMOTIVE,7.0,0
3000789,2017-08-15,7,AUTOMOTIVE,5.0,0
3000822,2017-08-15,8,AUTOMOTIVE,4.0,0
3000855,2017-08-15,9,AUTOMOTIVE,15.0,0


Wyliczamy średnią sprzedaż na daną datę

In [46]:
train_automotive2= train_automotive.groupby(['date'])['sales'].mean().to_frame()

In [47]:
train_automotive2.head()

,sales
date,
2013-01-01,0.000000
2013-01-02,4.722222
2013-01-03,2.981481
2013-01-04,3.129630
2013-01-05,6.333333


Dołączamy informacje o zmiennych i robimy one-hot encoding zmiennej locale

In [48]:
train_automotive_merged = train_automotive2.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(train_automotive_merged[['locale']]).toarray())

final_train_automotive = train_automotive_merged.join(encoder_df)

final_train_automotive.drop('locale', axis=1, inplace=True)

final_train_automotive.columns = ['date', 'sales','type','local_name','description','transferred','isLocal','isNational','isRegional','isNormalDay']


In [49]:
final_train_automotive.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNormalDay
0,2013-01-01,0.000000,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
1,2013-01-02,4.722222,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
2,2013-01-03,2.981481,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
3,2013-01-04,3.129630,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
4,2013-01-05,6.333333,Work Day,Ecuador,Recupero puente Navidad,False,0.0,1.0,0.0,0.0


Tworzymy zmienną dayofweek

In [50]:
final_train_automotive['dayofweek'] = pd.DatetimeIndex(final_train_automotive['date']).dayofweek + 1

In [51]:
final_train_automotive.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNormalDay,dayofweek
0,2013-01-01,0.000000,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
1,2013-01-02,4.722222,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
2,2013-01-03,2.981481,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,4
3,2013-01-04,3.129630,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,5
4,2013-01-05,6.333333,Work Day,Ecuador,Recupero puente Navidad,False,0.0,1.0,0.0,0.0,6


Dodajemy oil jako zmienną objaśniającą.

In [52]:
train_automotive_oil = final_train_automotive.merge(oil2,how="left",left_on=['date'],right_on=['date'])

In [53]:
train_automotive_oil.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
0,2013-01-01,0.000000,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2,NaN
1,2013-01-02,4.722222,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3,93.14
2,2013-01-03,2.981481,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,4,92.97
3,2013-01-04,3.129630,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,5,93.12
4,2013-01-05,6.333333,Work Day,Ecuador,Recupero puente Navidad,False,0.0,1.0,0.0,0.0,6,NaN


Interpolacja

In [54]:
train_automotive_oil.interpolate(method ='linear', limit_direction ='backward', inplace=True)

In [55]:
train_automotive_oil.tail(10)

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
1704,2017-08-06,10.796296,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,7,49.436667
1705,2017-08-07,6.574074,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1,49.370000
1706,2017-08-08,6.055556,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,2,49.070000
1707,2017-08-09,5.814815,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3,49.590000
1708,2017-08-10,5.796296,Holiday,Ecuador,Primer Grito de Independencia,True,0.0,1.0,0.0,0.0,4,48.540000
1709,2017-08-11,8.166667,Transfer,Ecuador,Traslado Primer Grito de Independencia,False,0.0,1.0,0.0,0.0,5,48.810000
1710,2017-08-12,7.462963,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,6,48.403333
1711,2017-08-13,8.907407,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,7,47.996667
1712,2017-08-14,5.407407,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1,47.590000
1713,2017-08-15,6.240741,Holiday,Riobamba,Fundacion de Riobamba,False,1.0,0.0,0.0,0.0,2,47.570000


Dzielimy próbkę train na treningową i testową.

In [56]:
train2 = train_automotive_oil.loc[(train_automotive_oil['date']<'2016-06-01')]
test2 = train_automotive_oil.loc[(train_automotive_oil['date']>='2016-06-01')].reset_index(drop=True)

In [57]:
test2.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
0,2016-06-01,6.425926,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3,49.07
1,2016-06-02,5.740741,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,4,49.14
2,2016-06-03,5.888889,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,5,48.69
3,2016-06-04,9.000000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,6,49.03
4,2016-06-05,11.185185,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,7,49.37


In [58]:
train2_date = train2.copy()
train2.drop(["description","date","type","local_name","transferred"],axis=1,inplace=True)
test2.drop(["description","date","type","local_name","transferred"],axis=1,inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15988\2658594707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train2.drop(["description","date","type","local_name","transferred"],axis=1,inplace=True)


Tworzymy model liniowy

In [59]:
X = train2.drop(['sales'],axis=1)
Y = train2['sales']

In [60]:
model = sm.OLS(Y,X).fit()

In [61]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     239.3
Date:                Thu, 21 Apr 2022   Prob (F-statistic):          1.08e-179
Time:                        21:26:18   Log-Likelihood:                -2206.9
No. Observations:                1264   AIC:                             4426.
Df Residuals:                    1258   BIC:                             4457.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
isLocal         6.2705      0.206     30.386      

In [62]:
test2.head()

,sales,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
0,6.425926,0.0,0.0,0.0,1.0,3,49.07
1,5.740741,0.0,0.0,0.0,1.0,4,49.14
2,5.888889,0.0,0.0,0.0,1.0,5,48.69
3,9.000000,0.0,0.0,0.0,1.0,6,49.03
4,11.185185,0.0,0.0,0.0,1.0,7,49.37


In [63]:
train2.head()

,sales,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
0,0.000000,0.0,1.0,0.0,0.0,2,93.140000
1,4.722222,0.0,0.0,0.0,1.0,3,93.140000
2,2.981481,0.0,0.0,0.0,1.0,4,92.970000
3,3.129630,0.0,0.0,0.0,1.0,5,93.120000
4,6.333333,0.0,1.0,0.0,0.0,6,93.146667


In [64]:
test2_drop = test2.drop(['sales'],axis=1)
Y_test = test2['sales']

In [65]:
test2.head()

,sales,isLocal,isNational,isRegional,isNormalDay,dayofweek,dcoilwtico
0,6.425926,0.0,0.0,0.0,1.0,3,49.07
1,5.740741,0.0,0.0,0.0,1.0,4,49.14
2,5.888889,0.0,0.0,0.0,1.0,5,48.69
3,9.000000,0.0,0.0,0.0,1.0,6,49.03
4,11.185185,0.0,0.0,0.0,1.0,7,49.37


In [66]:
Y_pred = model.predict(test2_drop)

Policzmy MSE.

In [67]:
mean_squared_error(Y_test,Y_pred)

2.775696197477054